In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
import os
import random
import numpy as np
import time
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [3]:
import os
import keras_ocr
import torchvision.models as models

In [4]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
train_transforms = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                                 transforms.RandomRotation(degrees=5),
                                                 transforms.ColorJitter(),
                                                 #transforms.RandomHorizontalFlip(),
                                                 transforms.CenterCrop(size=224),	# Image net standards
                                                 #transforms.Resize((112,112)),
                                                 #transforms.Resize((224,224)),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize([0.5, 0.5, 0.5],
                                                 [0.5, 0.5, 0.5])
                                             ])

test_transforms = transforms.Compose([
                                        #transforms.CenterCrop((224, 224)),
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
                                    ])

In [6]:
train_data = datasets.ImageFolder(r"C:\Users\Jean-Michel\Downloads\my_data_aug\my_data_aug\train", train_transforms)
test_data = datasets.ImageFolder(r"C:\Users\Jean-Michel\Downloads\my_data_aug\my_data_aug\test", test_transforms)


In [7]:
n_train_examples = int(len(train_data)*0.8)
n_valid_examples = n_test_examples = len(train_data) - n_train_examples

train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples])


In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


Number of training examples: 4844
Number of validation examples: 1212
Number of testing examples: 1457


In [9]:
BATCH_SIZE = 32
train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

device = torch.device('cuda')

In [11]:
def train(model, device, iterator, optimizer, criterion):  
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for (x, y) in iterator:
        data = ocr(x)
        data = data.to(device)
        y = y.to(device)
        optimizer.zero_grad()  
        fx = model(data).to(device)
        loss = criterion(fx, y)
        acc = calculate_accuracy(fx, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, device, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:
            data = ocr(x)
            data = data.to(device)
            y = y.to(device)
            fx = model(data).to(device)
            loss = criterion(fx, y)
            acc = calculate_accuracy(fx, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [18]:
def ocr(x):
    images = []
    n_images = x.shape[0] 
    for i in range(n_images):
        im = x[0].cpu().permute(1,2,0)
        im = im*0.5+0.5
        im = im.numpy()
        im = im*255
        images.append(im)
    prediction_groups = pipeline.recognize(images)
    word_list = ['signal', 'stop', 'ahead', 'speed', 'limit', '15', '30', '35', '45', '55', '65', 'right', 'left', 'lane', 'ends', 'do', 'not', 'enter', 'yield']
    data = torch.zeros(n_images,len(word_list))
    for i in range(n_images):
        n_words = len(prediction_groups[i])
        for j in range(n_words):
            word = prediction_groups[i][j][0]
            for k in range(len(word_list)):
                if(word == word_list[k]):
                    data[i,k] = 1
    return data

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
D_in, H, D_out = 19, 100, 18

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)


In [19]:
model = model.to(device)

for param in model.parameters():
	param.requires_grad = True

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.

t_train_acc = []
t_val_acc = []
t_train_loss = []
t_val_loss = []


In [20]:
EPOCHS = 50
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ocr.pt')
best_valid_loss = float('inf')
pipeline = keras_ocr.pipeline.Pipeline()

for epoch in range(EPOCHS):

    start = time.time()
    
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)


    t_train_loss.append(train_loss)
    t_train_acc.append(train_acc)

    t_val_acc.append(valid_acc)
    t_val_loss.append(valid_loss)
   

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    T = time.time() - start

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | T: {T:0.2f}s|')

Looking for C:\Users\Jean-Michel\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\Jean-Michel\.keras-ocr\crnn_kurapan.h5
4/4 [==============================] - 2s 447ms/step
| Epoch: 01 | Train Loss: 2.778 | Train Acc: 10.75% | Val. Loss: 2.709 | Val. Acc: 13.30% | T: 1777.40s|
4/4 [==============================] - 2s 459ms/step
| Epoch: 02 | Train Loss: 2.699 | Train Acc: 12.16% | Val. Loss: 2.685 | Val. Acc: 12.97% | T: 1744.61s|
1/3 [=========>....................] - ETA: 1s

KeyboardInterrupt: 